# 김기범 전치리 코드 테스트

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('../data/train.csv')

train_df.shape

(45678, 5)

# 1. add predefined news category

train_df['text'][:10]

train_df['input_text'][:10] + " [SEP] " + train_df['predefined_news_category'][:10]

# 2. AI HUB 뉴스 기사 데이터 가져오기

from glob import glob

ai_hub_017 = glob('../data/ai_hub_017/*.json')
ai_hub_017

import json

df = pd.DataFrame()

for path in ai_hub_017:
    try:
        with open(path, 'r') as f:
            js = json.loads(f.read())
    except Exception as e:
        print(e)

    new_df = pd.json_normalize(js['data'])

    df = pd.concat([df, new_df], axis=0)

df.shape

df.columns

df.head()

df['doc_class.code'].unique()

use_classes = ['정치', '경제', '사회', '문화', '스포츠', 'IT과학', '국제']

df2 = df[(df['doc_class.code'] != '지역') & (df['doc_class.code'] != '기타')][['doc_id', 'doc_title', 'doc_class.code']]
df2.shape

df2.drop_duplicates()

df2.shape

df2['doc_class.code'].unique()

df2['doc_class.code'] = df2['doc_class.code'].replace('국제', '세계')
df2['doc_class.code'] = df2['doc_class.code'].replace('문화', '생활문화')

df2['doc_class.code'].unique()

train_df.columns, df2.columns

label2num = {
    label: i for i, label in enumerate(['정치', '경제', '사회', '생활문화', '세계', 'IT과학', '스포츠'])
}
label2num

df2.columns = ['ID', 'input_text', 'label_text']
df2['target'] = df2['label_text'].apply(lambda x: label2num[x])

df2.columns

print(pd.concat([train_df, df2], axis=0).shape)
print(pd.concat([train_df, df2], axis=0).drop_duplicates(subset=['input_text']).shape)
print(pd.concat([train_df, df2], axis=0).drop_duplicates(subset=['input_text', 'target']).shape)

# df2.to_csv('../data/ai_hub_017.csv', index=False)

# 3. AI HUB 낚시 기사성 뉴스 데이터

ai_hub_146_val_path_list = glob("../data/ai_hub_146_val/*/*.json")
len(ai_hub_146_val_path_list)

id = []
category = []
title = []

for path in ai_hub_146_val_path_list:
    new_df = pd.read_json(path)

    id.append(new_df.loc['newsID']['sourceDataInfo'])
    category.append(new_df.loc['newsCategory']['sourceDataInfo'])
    title.append(new_df.loc['newsTitle']['sourceDataInfo'])

df = pd.DataFrame()
df['ID'] = id
df['input_text'] = category
df['label_text'] = title

df.shape

df.columns = ['ID', 'label_text', 'input_text']
df['label_text'].unique()

df['label_text'] = df['label_text'].replace(r'&', '', regex=True)
df['label_text'].unique()

df2['target'] = df2['label_text'].apply(lambda x: label2num[x])
df2.head()

# df2.to_csv('../data/ai_hub_146_val.csv', index=False)

# 4. AI HUB 데이터끼리 합치기

ai_hub_017 = pd.read_csv('../data/ai_hub_017.csv')
ai_hub_146 = pd.read_csv('../data/ai_hub_146_val.csv')
ai_hub = pd.concat([ai_hub_017, ai_hub_146], axis=0)

ai_hub.shape

ai_hub.drop_duplicates(subset=['input_text', 'target'], inplace=True)
ai_hub.shape

ai_hub[ai_hub.duplicated(subset=['input_text'], keep=False)].sort_values(by='input_text')

del_duplicate_index = [29288, 29764, 29406, 29750, 29442, 29773, 54108, 29626, 29450, 10340, 25707, 42812, 11774, 29751,
                       29443, 30159, 30137, 29813, 24469, 40823, 29762, 82929, 79340, 29810, 30135, 29801, 29752, 18799, 30136]

ai_hub.drop(del_duplicate_index, axis=0, inplace=True)
ai_hub.shape

ai_hub['target'].value_counts()

test_count = [722, 1348, 3701, 1369, 835, 554, 578]

for idx, count in enumerate(test_count):
    print(idx, ">", round(count / 9107 * 100, 1))

(ai_hub['target'].value_counts(normalize=True) * 100).sort_index()

# ai_hub.to_csv('../data/ai_hub_original.csv', index=False)

# ai_hub 분포 맞추기

# 5. 최종 데이터셋 생성

train_no_g2p = pd.read_csv('../data/train_No_G2P.csv')
label_error_detecting = pd.read_csv('../data/label_error_detecting.csv')

train_no_g2p.shape, label_error_detecting.shape

label_error_detecting.columns

label_error_detecting.tail()

label_error_detecting['정답'].value_counts()

train_no_g2p.drop(label_error_detecting[label_error_detecting['정답'] == False].index, axis=0, inplace=True)
train_no_g2p.shape, 47049 - 193

train_no_g2p[train_no_g2p.duplicated(subset=['ID', 'input_text'])].shape, 1365 + 198

# 6. 데이터셋 뒤섞기

train_df.head()

train_df.sample(frac=1, random_state=42, axis=0).head()

# 7. Korean Easy Data Augmentation

In [3]:
!pip install koeda

In [4]:
from koeda import EDA

In [5]:
eda = EDA(
    morpheme_analyzer="Okt", alpha_sr=0.3, alpha_ri=0.3, alpha_rs=0.3, prob_rd=0.3
)

In [7]:
for s in train_df['text'][:10]:
    change = eda(s)

    print("="*10)
    print("before >>", s)
    print("after >>", change)

before >> 유튜브 내달 2일까지 크리에이터 지원 공간 운영
after >> 유튜브운영내달 2일까지 크리에이터 공간 지원
before >> 어버이날 막따가 흐려저…남부지방 여튼 황사
after >> 막따가 …지방 튼 황사
before >> 내년부터 국가RD 평가 때 논문건수는 반영 않는다
after >> 내년부터 국가RD 평가 때 논문건수는 감응 않는다
before >> 김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
after >> 김명자 신임 회장 원로와 모을
before >> 회새긴간 작까 김동시 걍심꼬백 뜽 새 소설집 뚜권 출간
after >> 작까 동시 뜽 새 소설집 뚜 출간
before >> 야외서 생방송 하세요…액션캠 전용 요금제 잇따라
after >> 야외서 생방송하세요 액션캠 전용 요금제…잇따라
before >> 월드컵 태극전사 16강 전초기지 레오강 입성종합
after >> 월드컵 16강 전초기지 강 입성
before >> 미세먼지 속 출근길
after >> 미세먼지 정신 출근길
before >> 왓츠앱稅 230원에 성난 레바논 민심…총리사퇴로 이어져종합2보
after >> 왓츠앱稅 230원에 성난 레바논 거부 민심… 사절 총리 관리 사퇴로 정경 이어져종합2보
before >> 베트남 경제 고성장 지속…2분기 GDP 6.71% 성장
after >> 베트남 경제 고성장 지속…2분 성기 기 GDP 6.71% 급속도 고생 성장


In [13]:
train_df['text'][:5]

0         유튜브 내달 2일까지 크리에이터 지원 공간 운영
1            어버이날 막따가 흐려저…남부지방 여튼 황사
2        내년부터 국가RD 평가 때 논문건수는 반영 않는다
3    김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4     회새긴간 작까 김동시 걍심꼬백 뜽 새 소설집 뚜권 출간
Name: text, dtype: object

In [14]:
from koeda import RD

func = RD("Okt")
sen_after_RD = func(train_df['text'].to_list(), 0.1)

sen_after_RD[:5]

['유튜브 까지 크리에이터 공간 운영',
 '어버이날 막따가 흐려저남부지방 여튼 황사',
 '내년부터 국가RD 평가 때 논문건수 반영 않는다',
 '김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것',
 '회새긴간 작까 김동시 걍심꼬백 뜽 새 소설집 권 출간']

In [15]:
from koeda import RS

func = RS("Okt")
sen_after_RS = func(train_df['text'].to_list(), 0.1)

sen_after_RS[:5]

['유튜브크리에이터내달 2일까지 지원 공간 운영',
 '막따가 흐려저…남부지방 여튼어버이날황사',
 '부터 국가RD 평가 때 논문건수는 반영내년않는다',
 '김명자 신임 회장 원로와 젊은 과학자과총지혜 모을 것',
 '회새긴간 작까 김동시 걍심꼬백간뜽 새 소설집 뚜권 출']

In [48]:
train_df['findall'] = train_df['text'].str.findall(pat='[^A-Za-z0-9가-힣\s]{1,9999}')

In [50]:
train_df.columns

Index(['ID', 'text', 'target', 'url', 'date', 'findall'], dtype='object')

In [66]:
train_df['findall']

0           []
1          […]
2           []
3           []
4           []
         ...  
45673      [·]
45674      […]
45675    [...]
45676       []
45677    [...]
Name: findall, Length: 45678, dtype: object

In [77]:
special_words = set([word for row in train_df['findall'] for word in row])

In [93]:
train_df = pd.read_csv('../data/train_spelling.csv')

train_df['text']

0              유튜브 내달 2일까지 크리에이터 지원 공간 운영
1                 어버이날 맑다가 흐려져…남부지방 옅은 황사
2             내년부터 국가RD 평가 때 논문건수는 반영 않는다
3         김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4          회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간
                       ...               
45673        사우디 아람코 주요 석유시설·유전 드론 공격에 화재
45674      박기원 감독 눈치 보지 말고…비예나 눈치 본 건 아닌데
45675    아시안게임 첫 경기 앞둔 라건아 인도네시아 팬이라도 ...
45676       트럼프 미중 무역협상 1단계 합의 서명식 가질 것종합
45677    극적 역전승 도로공사 GS칼텍스 꺾고 2년 연속 챔프...
Name: text, Length: 45678, dtype: object

In [94]:
for word in sorted(list(special_words), key=lambda x: len(x), reverse=True):
    train_df['text'] = train_df['text'].str.replace(word, f" {word} ")

train_df['text'] = train_df['text'].str.replace('[\s]{2, 9999}', " ")
train_df['text']

0                        유튜브 내달 2일까지 크리에이터 지원 공간 운영
1                         어버이날 맑다가 흐려져 … 남부지방 옅은 황사
2                       내년부터 국가RD 평가 때 논문건수는 반영 않는다
3                   김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4                    회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간
                            ...                    
45673                사우디 아람코 주요 석유시설 · 유전 드론 공격에 화재
45674              박기원 감독 눈치 보지 말고 … 비예나 눈치 본 건 아닌데
45675    아시안게임 첫 경기 앞둔 라건아 인도네시아 팬이라도    .  .   .  
45676                 트럼프 미중 무역협상 1단계 합의 서명식 가질 것종합
45677    극적 역전승 도로공사 GS칼텍스 꺾고 2년 연속 챔프   .  .   .  
Name: text, Length: 45678, dtype: object

# 8. label error 잡기 위해 target에 따른 단어 빈도 수 보기

In [1]:
from collections import Counter